## Comments by Felix

- We are scaling Dummy variables, I'm not sure this is the right approach
- We are using fields such as "area_order" and "secion_order" as scaled continous variables, when we already have this information in the dummy variables "area_name" and "section_name"
- Shall we drop user_exercise_id?
- nr_task seems important. we probably shouldn't just drop this. It is also needed for our predictions.

In [1]:
import pandas as pd
import numpy as np
import sklearn
import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [2]:
# Import data for training and prediction
train = pd.read_csv('data_for_training.csv')
predict = pd.read_csv('data_for_prediction.csv')
train.head()

,user_id,nr_tasks,created_at,time_obs,item_id,field,area_name,area_order,section_name,section_order,user_exercise_id,exercise_duration,exercise_time_s,performance_task
0,1,1,"Mon, 12 Apr 2021 17:50:58 -0400",2021-04-12,23,vocabulary,Speaking Strategies,20,Asking and Answering,130,2463,04:42 minutes,282,282.0
1,1,10,"Tue, 18 May 2021 16:37:25 -0400",2021-05-18,23,vocabulary,Speaking Strategies,20,Asking and Answering,130,2511,03:30 minutes,210,21.0
2,1,10,"Fri, 30 Apr 2021 15:28:38 -0400",2021-04-30,121,grammar,Numeral,80,Cardinal,500,2227,02:23 minutes,143,14.3
3,2,10,"Fri, 02 Apr 2021 05:03:39 -0400",2021-04-02,153,grammar,Main Clause,100,Satzklammer für Perfekt,820,526,57 seconds,57,5.7
4,2,10,"Mon, 12 Jul 2021 06:09:39 -0400",2021-07-12,124,grammar,Verb,90,Word Order,530,195,02:58 minutes,178,17.8


In [3]:
# Remove unnecessary columns
train = train.drop(['nr_tasks', 'exercise_duration', 'created_at'], axis = 1)

In [4]:
# One Hot Encode "field"
one_hot_field = pd.get_dummies(train['field'])
train = train.join(one_hot_field)
train = train.drop('field', axis = 1)

# One Hot Encode "area_name"
one_hot_area_name = pd.get_dummies(train['area_name'])
train = train.join(one_hot_area_name)
train = train.drop('area_name', axis = 1)

# One Hot Encode "section_name"
one_hot_section_name = pd.get_dummies(train['section_name'])
train = train.join(one_hot_section_name)
train = train.drop('section_name', axis = 1)

train.head()


,user_id,time_obs,item_id,area_order,section_order,user_exercise_id,exercise_time_s,performance_task,grammar,vocabulary,...,Subjunctive I,Subjunctive II,Temporal,Time,Word Formation,Word Order,Yes / No Interrogative (Entscheidungsfrage),Zero,n-Declension,with Prepositions
0,1,2021-04-12,23,20,130,2463,282,282.0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,2021-05-18,23,20,130,2511,210,21.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1,2021-04-30,121,80,500,2227,143,14.3,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2,2021-04-02,153,100,820,526,57,5.7,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2,2021-07-12,124,90,530,195,178,17.8,1,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
# Make sure that date is in datetime format
train['time_obs'] = pd.to_datetime(train['time_obs'])

# Transform date into string
train['time_string'] = train['time_obs'].dt.strftime('%Y%m%d')
train = train.drop('time_obs', axis = 1)
train.head()

,user_id,item_id,area_order,section_order,user_exercise_id,exercise_time_s,performance_task,grammar,vocabulary,Adjective,...,Subjunctive II,Temporal,Time,Word Formation,Word Order,Yes / No Interrogative (Entscheidungsfrage),Zero,n-Declension,with Prepositions,time_string
0,1,23,20,130,2463,282,282.0,0,1,0,...,0,0,0,0,0,0,0,0,0,20210412
1,1,23,20,130,2511,210,21.0,0,1,0,...,0,0,0,0,0,0,0,0,0,20210518
2,1,121,80,500,2227,143,14.3,1,0,0,...,0,0,0,0,0,0,0,0,0,20210430
3,2,153,100,820,526,57,5.7,1,0,0,...,0,0,0,0,0,0,0,0,0,20210402
4,2,124,90,530,195,178,17.8,1,0,0,...,0,0,0,0,1,0,0,0,0,20210712


In [6]:
# Split the dataset into the training dataset and the test dataset.
def split_data(X, y, split_coeff):
    N, _ = X.shape 
    train_size = int(split_coeff * N) 
    X_train = X[:train_size] 
    y_train = y[:train_size]
    X_test = X[train_size:] 
    y_test = y[train_size:]
    
    return X_train, y_train, X_test, y_test

# Define X and y and call the split fct
y = train['exercise_time_s']
X = train.drop('exercise_time_s', axis = 1)
X_train, y_train, X_test, y_test = split_data(X, y, 0.8) 

# check the size of the splitted dataset
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1545, 94)
Shape of y_train: (1545,)
Shape of X_test: (387, 94)
Shape of y_test: (387,)


In [7]:
# Standardize data
scale = StandardScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.transform(X_test)

# transform back into df
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_test.columns)

X_train_scaled.head()

,user_id,item_id,area_order,section_order,user_exercise_id,performance_task,grammar,vocabulary,Adjective,Adverb,...,Subjunctive II,Temporal,Time,Word Formation,Word Order,Yes / No Interrogative (Entscheidungsfrage),Zero,n-Declension,with Prepositions,time_string
0,-1.061928,-1.294574,-0.477486,-0.713702,0.925701,1.163375,-1.062120,1.062120,-0.358554,-0.297985,...,-0.062439,-0.036002,-0.025449,-0.257396,-0.244276,-0.044108,-0.076547,-0.025449,-0.036002,-0.714505
1,-1.061928,-1.294574,-0.477486,-0.713702,0.977439,-0.302061,-1.062120,1.062120,-0.358554,-0.297985,...,-0.062439,-0.036002,-0.025449,-0.257396,-0.244276,-0.044108,-0.076547,-0.025449,-0.036002,-0.275401
2,-1.061928,0.985212,-0.265590,0.958722,0.671323,-0.339679,0.941513,-0.941513,-0.358554,-0.297985,...,-0.062439,-0.036002,-0.025449,-0.257396,-0.244276,-0.044108,-0.076547,-0.025449,-0.036002,-0.639940
3,-0.756458,1.729632,-0.194958,2.405142,-1.162143,-0.387965,0.941513,-0.941513,-0.358554,-0.297985,...,-0.062439,-0.036002,-0.025449,-0.257396,-0.244276,-0.044108,-0.076547,-0.025449,-0.036002,-0.755930
4,-0.756458,1.055001,-0.230274,1.094324,-1.518919,-0.320028,0.941513,-0.941513,-0.358554,-0.297985,...,-0.062439,-0.036002,-0.025449,-0.257396,4.093729,-0.044108,-0.076547,-0.025449,-0.036002,0.528240


In [8]:
############
# MODEL 1: #
############
# Linear regression without accounting for different agents

# fit the model
reg = LinearRegression().fit(X_train_scaled, y_train)

# print performance of regression
print('Lin Reg performance on test set:', reg.score(X_test_scaled, y_test))

# predict future students performance
predict_lin_reg = reg.predict(X_test_scaled)

# Calculate the absolute errors
errors_lin_reg = abs(predict_lin_reg - y_test)

# Print out the mean absolute error (mae)
print('Lin Reg Mean Absolute Error on test set:', round(np.mean(errors_lin_reg), 2))

Lin Reg performance on test set: 0.5785564518347345
Lin Reg Mean Absolute Error on test set: 78.37


In [9]:
############
# MODEL 2: #
############
# Random Forest without accounting for different agents

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

# Train the model on training data
rf.fit(X_train_scaled, y_train);

# Use the forest's predict method on the test data
predict_rf = rf.predict(X_test_scaled)

# Calculate the absolute errors
errors_rf = abs(predict_rf - y_test)

# Print out the mean absolute error (mae)
print('Random Forest Mean Absolute Error on test set:', round(np.mean(errors_rf), 2))

Random Forest Mean Absolute Error on test set: 22.94


In [11]:
############
# MODEL 3: #
############
# NN without accounting for different agents
from numpy import asarray
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense

# create model
model = Sequential()
model.add(Dense(20, input_dim=94, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, kernel_initializer='he_uniform'))
model.compile(loss='mae', optimizer='adam')

# fit the model on all data
model.fit(X_train_scaled, y_train, verbose=0, epochs=100)

# predict
predict_NN = model.predict(X_test_scaled)
# ugly df to array transformations
predict_NN = pd.DataFrame(predict_NN)
predict_NN2 = pd.DataFrame.to_numpy(predict_NN)
predict_NN2 = pd.DataFrame(predict_NN2)
y_test = pd.DataFrame(y_test)
y_test2 = pd.DataFrame.to_numpy(y_test)
y_test2 = pd.DataFrame(y_test2)

# Calculate the absolute errors
errors_NN = abs(predict_NN2 - y_test2)

# Print out the mean absolute error (mae)
print('Neural Net Mean Absolute Error on test set:', round(np.mean(errors_NN), 2))

Neural Net Mean Absolute Error on test set: 0    56.94
dtype: float64


In [23]:
############
# MODEL 4: #
############
# KNN without accounting for different agents
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=3)

# fit the model
knn_model.fit(X_train_scaled, y_train)

# predict
predict_knn = knn_model.predict(X_test_scaled)
# ugly df to array transformations
predict_knn = pd.DataFrame(predict_knn)
predict_knn2 = pd.DataFrame.to_numpy(predict_knn)
predict_knn2 = pd.DataFrame(predict_knn2)
y_test = pd.DataFrame(y_test)
y_test2 = pd.DataFrame.to_numpy(y_test)
y_test2 = pd.DataFrame(y_test2)

# Calculate the absolute errors
errors_knn = abs(predict_knn - y_test2)

# Print out the mean absolute error (mae)
print('k-nearest neighbors Mean Absolute Error on test set:', round(np.mean(errors_knn), 2))

k-nearest neighbors Mean Absolute Error on test set: 0    58.85
dtype: float64
